# ETL com Google Cloud Storage, BigQuery e Python

## Visão Geral
Este notebook demonstra um fluxo completo de ETL (Extract, Transform, Load) integrando:
- **Google Cloud Storage (GCS)**: Para armazenamento de arquivos
- **BigQuery**: Para consultas e análise de dados em larga escala
- **Python**: Para processamento e transformação de dados

### 🔐 Segurança com Colab Secrets
Este notebook usa **Colab Secrets** para manter suas credenciais seguras. No Colab:

1. Clique no ícone 🔑 **Secrets** à esquerda
2. Adicione dois secrets:
   - `GCP_PROJECT_ID` = seu Google Cloud Project ID
   - `GCS_BUCKET_NAME` = seu bucket GCS

## Fases do Projeto
1. **EXTRACT**: Autenticação e extração de dados do GCS e BigQuery
2. **TRANSFORM**: Análise e processamento de dados (cálculo de atrasos)
3. **LOAD**: Armazenamento dos dados processados no BigQuery e GCS

## FASE 1: EXTRACT - Autenticação e Conexão aos Serviços

Nesta seção, vamos autenticar no Google Cloud e conectar aos serviços necessários (GCS e BigQuery).

In [1]:
# Autenticar no Google Colab para acessar recursos do Google Cloud
from google.colab import auth

auth.authenticate_user()

In [2]:
from google.cloud import storage

In [3]:
from google.colab import userdata

# Colab Secrets
project_id = userdata.get('GCP_PROJECT_ID')
bucket_name = userdata.get('GCS_BUCKET_NAME')
print("✅ Credenciais carregadas do Colab Secrets.")

file_name = 'BR.json'

# Verificar se as variáveis estão configuradas
if not project_id or not bucket_name:
    raise ValueError(
        "❌ Variáveis não configuradas!\n"
        "Para Colab, use Secrets (🔑 ícone à esquerda):\n"
        "  GCP_PROJECT_ID = seu-project-id\n"
        "  GCS_BUCKET_NAME = seu-bucket-name"
    )

✅ Credenciais carregadas do Colab Secrets.


In [4]:
# Criar cliente GCS e acessar o arquivo no bucket
client_gcs = storage.Client(project = project_id)

# Obter referência do bucket
bucket = client_gcs.bucket(bucket_name)

# Obter referência do arquivo (blob) dentro do bucket
blob = bucket.blob(file_name)

In [5]:
# Download do arquivo do GCS como string (texto)
file_content_str = blob.download_as_text()

print(f"Conteúdo do arquivo {file_name}")
print(file_content_str)

Conteúdo do arquivo BR.json
[{"date":"2016-01-01","localName":"Confraternização Universal","name":"New Year's Day","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Public"]},{"date":"2016-02-08","localName":"Carnaval","name":"Carnival","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Bank","Optional"]},{"date":"2016-02-09","localName":"Carnaval","name":"Carnival","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Bank","Optional"]},{"date":"2016-03-25","localName":"Sexta-feira Santa","name":"Good Friday","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Public"]},{"date":"2016-03-27","localName":"Domingo de Páscoa","name":"Easter Sunday","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Public"]},{"date":"2016-04-21","localName":"Dia de Tiradentes","name":"Tiradentes","countryCode":"

In [6]:
file_content_str

'[{"date":"2016-01-01","localName":"Confraternização Universal","name":"New Year\'s Day","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Public"]},{"date":"2016-02-08","localName":"Carnaval","name":"Carnival","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Bank","Optional"]},{"date":"2016-02-09","localName":"Carnaval","name":"Carnival","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Bank","Optional"]},{"date":"2016-03-25","localName":"Sexta-feira Santa","name":"Good Friday","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Public"]},{"date":"2016-03-27","localName":"Domingo de Páscoa","name":"Easter Sunday","countryCode":"BR","fixed":false,"global":true,"counties":null,"launchYear":null,"types":["Public"]},{"date":"2016-04-21","localName":"Dia de Tiradentes","name":"Tiradentes","countryCode":"BR","fixed":false,"global"

In [7]:
# Converter a string JSON para objeto Python
import json

dados_feriados = json.loads(file_content_str)

# Exibir os dados extraídos
dados_feriados

[{'date': '2016-01-01',
  'localName': 'Confraternização Universal',
  'name': "New Year's Day",
  'countryCode': 'BR',
  'fixed': False,
  'global': True,
  'counties': None,
  'launchYear': None,
  'types': ['Public']},
 {'date': '2016-02-08',
  'localName': 'Carnaval',
  'name': 'Carnival',
  'countryCode': 'BR',
  'fixed': False,
  'global': True,
  'counties': None,
  'launchYear': None,
  'types': ['Bank', 'Optional']},
 {'date': '2016-02-09',
  'localName': 'Carnaval',
  'name': 'Carnival',
  'countryCode': 'BR',
  'fixed': False,
  'global': True,
  'counties': None,
  'launchYear': None,
  'types': ['Bank', 'Optional']},
 {'date': '2016-03-25',
  'localName': 'Sexta-feira Santa',
  'name': 'Good Friday',
  'countryCode': 'BR',
  'fixed': False,
  'global': True,
  'counties': None,
  'launchYear': None,
  'types': ['Public']},
 {'date': '2016-03-27',
  'localName': 'Domingo de Páscoa',
  'name': 'Easter Sunday',
  'countryCode': 'BR',
  'fixed': False,
  'global': True,
  'cou

### Agora vamos trabalhar com dados no BigQuery
Vamos conectar ao BigQuery e extrair dados de pedidos para análise.

In [8]:
from google.cloud import bigquery

In [9]:
client_bq = bigquery.Client(project = project_id)

In [10]:
# Consulta inicial: extrair dados básicos de pedidos
# Obtemos informações sobre status, datas de compra e entrega
consulta_pedidos = """
SELECT
order_id, order_status, order_purchase_timestamp,
order_estimated_delivery_date, order_delivered_customer_date
FROM `{project_id}.olist_dataset.orders`
""".format(project_id=project_id)

In [11]:
query_job = client_bq.query(consulta_pedidos)

In [12]:
pedidos = query_job.to_dataframe()
pedidos

,order_id,order_status,order_purchase_timestamp,order_estimated_delivery_date,order_delivered_customer_date
0,a2e4c44360b4a57bdff22f3a4630c173,approved,2017-02-06 20:18:17+00:00,2017-03-01 00:00:00+00:00,NaT
1,132f1e724165a07f6362532bfb97486e,approved,2017-04-25 01:25:34+00:00,2017-05-22 00:00:00+00:00,NaT
2,809a282bbd5dbcabb6f2f724fca862ec,canceled,2016-09-13 15:24:19+00:00,2016-09-30 00:00:00+00:00,NaT
3,e5215415bb6f76fe3b7cb68103a0d1c0,canceled,2016-10-22 08:25:27+00:00,2016-10-24 00:00:00+00:00,NaT
4,71303d7e93b399f5bcd537d124c0bcfa,canceled,2016-10-02 22:07:52+00:00,2016-10-25 00:00:00+00:00,NaT
...,...,...,...,...,...
99436,4cccc0d35e7c7a0dc766ad3c4043e33e,unavailable,2018-08-10 09:32:32+00:00,2018-08-15 00:00:00+00:00,NaT
99437,897b4da63b6edde1a33a9fb7caf1dd10,unavailable,2018-07-30 07:38:21+00:00,2018-08-16 00:00:00+00:00,NaT
99438,93881917b8e0f2bf11eec7abbbfe43ec,unavailable,2018-08-11 21:38:00+00:00,2018-08-21 00:00:00+00:00,NaT
99439,4bd0d8aa4756f78245bd56015d4ddcc0,unavailable,2018-08-11 11:56:24+00:00,2018-08-27 00:00:00+00:00,NaT


## FASE 2: TRANSFORM - Análise e Processamento de Dados

Nesta seção, vamos criar uma análise de atrasos nas entregas. Para cada pedido entregue com atraso,
calculamos quantos dias após a data estimada a entrega ocorreu.

In [13]:
# Consulta para calcular atrasos nas entregas
# Usa DATE_DIFF para calcular a diferença em dias entre data real e estimada
# Filtra apenas pedidos que tiveram atraso (data real > data estimada)
consulta_atrasos = f"""
SELECT order_id, order_estimated_delivery_date,
order_delivered_customer_date,
DATE_DIFF(order_delivered_customer_date, order_estimated_delivery_date, DAY) AS atraso_medio_dias
FROM `{project_id}.olist_dataset.orders`
WHERE
    order_delivered_customer_date IS NOT NULL
    AND order_estimated_delivery_date IS NOT NULL
    AND order_delivered_customer_date > order_estimated_delivery_date
ORDER BY atraso_medio_dias DESC
"""

results = client_bq.query(consulta_atrasos)
df_atraso = results.to_dataframe()

print(f"Total de pedidos com atraso: {len(df_atraso)}")
print(f"\nEstatísticas de atraso (dias):")
print(df_atraso['atraso_medio_dias'].describe())

Total de pedidos com atraso: 7827

Estatísticas de atraso (dias):
count       7827.0
mean      8.867254
std      13.949441
min            0.0
25%            1.0
50%            5.0
75%           11.0
max          188.0
Name: atraso_medio_dias, dtype: Float64


In [14]:
results = client_bq.query(consulta_atrasos)
df_atraso = results.to_dataframe()
df_atraso

,order_id,order_estimated_delivery_date,order_delivered_customer_date,atraso_medio_dias
0,1b3190b2dfa9d789e1f14c05b647a14a,2018-03-15 00:00:00+00:00,2018-09-19 23:24:07+00:00,188
1,ca07593549f1816d26a572e06dc1eab6,2017-03-22 00:00:00+00:00,2017-09-19 14:36:39+00:00,181
2,47b40429ed8cce3aee9199792275433f,2018-01-19 00:00:00+00:00,2018-07-13 20:51:31+00:00,175
3,2fe324febf907e3ea3f2aa9650869fa5,2017-04-05 00:00:00+00:00,2017-09-19 17:00:07+00:00,167
4,285ab9426d6982034523a855f55a885e,2017-04-06 00:00:00+00:00,2017-09-19 14:00:04+00:00,166
...,...,...,...,...
7822,eb2e56877e1f053dedc857408122a7b4,2018-08-30 00:00:00+00:00,2018-08-30 01:20:34+00:00,0
7823,124223dc899eca6ac54187a54f1e3632,2018-08-30 00:00:00+00:00,2018-08-30 21:51:12+00:00,0
7824,823e4c9d908f5ec1cad58e1e3e7d716d,2018-08-31 00:00:00+00:00,2018-08-31 02:21:48+00:00,0
7825,d779224365f07d6953c45ef342c824e6,2018-08-31 00:00:00+00:00,2018-08-31 00:48:33+00:00,0


## FASE 3: LOAD - Armazenamento de Dados Processados

Nesta seção, vamos:
1. Definir um schema para a tabela no BigQuery
2. Carregar os dados transformados em uma nova tabela
3. Exportar os dados para um arquivo CSV no GCS

In [15]:
# Definir o caminho da tabela no BigQuery
caminho = f'{project_id}.olist_dataset.pedido_atrasos'

# Definir o schema (estrutura) da tabela
schema = [
    bigquery.SchemaField("order_id", "STRING"),
    bigquery.SchemaField("order_estimated_delivery_date", "TIMESTAMP"),
    bigquery.SchemaField("order_delivered_customer_date", "TIMESTAMP"),
    bigquery.SchemaField("atraso_medio_dias", "INTEGER")
]

# Configurar o job de carregamento
# WRITE_APPEND adiciona dados à tabela existente (cria se não existir)
job_config = bigquery.LoadJobConfig(
    schema = schema,
    write_disposition = "WRITE_APPEND"
)

# Carregar o DataFrame para o BigQuery
job = client_bq.load_table_from_dataframe(
    df_atraso,
    caminho,
    job_config = job_config
)

# Aguardar conclusão do carregamento
job.result()

print("Tabela carregada com sucesso!")

Tabela carregada com sucesso!


In [16]:
# Exportar o DataFrame para um arquivo CSV local
df_atraso.to_csv('dados.csv')

In [17]:
# O bucket já está definido nas variáveis de ambiente da célula anterior
# Configurar o caminho de destino no GCS
caminho_gcs = 'dados/dados_atrasos.csv'

In [18]:
bucket = client_gcs.get_bucket(bucket_name)

In [19]:
# Upload do arquivo CSV para o GCS
blob = bucket.blob(caminho_gcs)
blob.upload_from_filename('dados.csv')

print(f"✅ Arquivo enviado para GCS")

✅ Arquivo enviado para GCS
